## **Testing the Model**


### Model: **bart-large-mnli**

In [2]:
import torch
print(torch.__version__)

2.8.0+cpu


**Zero-Shot avec Pipeline**

To provide a simple, "one-liner" interface for common natural language processing (NLP) tasks. It's designed for ease of use and quick prototyping.

It automatically handles all the steps for you:

- Preprocessing: Tokenizes your text (converts words to numbers).

- Model Inference: Passes the tokens through the model.

- Post-processing: Takes the model's raw output and converts it into a human-readable format (e.g., softmax scores, labels).

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline


pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", clean_up_tokenization_spaces=True)

Device set to use cpu


In [4]:
sequence_to_classify = "you are all I need"
candidate_labels = ['love', 'mocking', 'flirting']
pipe(sequence_to_classify, candidate_labels)

{'sequence': 'you are all I need',
 'labels': ['love', 'flirting', 'mocking'],
 'scores': [0.5815867185592651, 0.2621539235115051, 0.15625937283039093]}

**Direct Loading**

To give the user full control over the entire process. This is necessary for advanced use cases.

What it does:

+ It breaks the pipeline's magic into its individual, explicit steps.

+ You manually handle:

    - Tokenization

    - Passing inputs to the model

    - Interpreting the model's raw output (logits)

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

In [10]:
print(model.config.label2id)

{'contradiction': 0, 'entailment': 2, 'neutral': 1}


In [12]:
# Your inputs
sequence_to_classify = "you are all I need"
candidate_labels = ['love', 'mocking', 'flirting']

# 1. Tokenize the sequence WITH the candidate labels (this is key for zero-shot)
# Use the `text_pair` argument explicitly for candidate labels
inputs = tokenizer(
    [sequence_to_classify] * len(candidate_labels),  # Repeat the sequence for each label
    text_pair=candidate_labels,  # Use candidate labels as the second input
    return_tensors="pt",
    padding=True,
    truncation=True
)

# 2. Run the model
with torch.no_grad():
    outputs = model(**inputs)

# 3. Get the predictions (logits)
logits = outputs.logits

# 4. You need to interpret these logits for a zero-shot task.
# The BERT NLI model gives output for entailment, contradiction, and neutral.
# For zero-shot, we typically take the "entailment" score for each label.
entailment_logits = logits[:, model.config.label2id['entailment']]  # Get the logits for the ENTAILMENT class

# 5. Apply softmax to get probabilities
probs = entailment_logits.softmax(dim=0)  # Use dim=1 for batch processing

# 6. Now you can print the results
scores = probs.tolist()
result = {"sequence": sequence_to_classify, "labels": candidate_labels, "scores": scores}
print(result)

{'sequence': 'you are all I need', 'labels': ['love', 'mocking', 'flirting'], 'scores': [0.5580359697341919, 0.27639028429985046, 0.16557373106479645]}
